In [1]:
import numpy as np
import netCDF4 as nc
import shapely.geometry as shpg
from shapely.ops import cascaded_union
import gdal

In [2]:
def get_buffered_ext(shape, buff_dist):
    """Create a buffer around a feature and return shape+buffer.
    """
    buffer = shape.buffer(buff_dist)
    total = cascaded_union(buffer, shape)
    return total

In [3]:
def pixelOffset2coord(rasterfn,xOffset,yOffset):
    """Change pixel coordinates to coordinates"""
    raster = gdal.Open(rasterfn)
    geotransform = raster.GetGeoTransform()
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    coordX = originX+pixelWidth*xOffset
    coordY = originY+pixelHeight*yOffset
    return coordX, coordY

In [4]:
def raster2array(rasterfn):
    """Convert a raster to numpy array"""
    raster = gdal.Open(rasterfn)
    geotransform = raster.GetGeoTransform()
    extent = [geotransform[0], geotransform[1], geotransform[3], geotransform[5]]
    band = raster.GetRasterBand(1)
    array = band.ReadAsArray()
    return array, extent

In [5]:
def getNoDataValue(rasterfn):
    """Get no data value from a raster"""
    raster = gdal.Open(rasterfn)
    band = raster.GetRasterBand(1)
    return band.GetNoDataValue()

In [12]:
def make_nc4(fpath, data, extent, location):
    """Compile a netCDF4 dataset"""
    #dataset.close()
    with nc.Dataset(fpath, 'w',  format='NETCDF4_CLASSIC') as dataset:
        
        # extract/calculate the data
        lats_out = np.arange(extent[2]-data.shape[0]*extent[3], extent[2], extent[3])
        lons_out = np.arange(extent[0]-data.shape[1]*extent[1], extent[1], extent[1])
        change_out = data[...,0]
        unc_out = data[...,1]
        mask_out = data[...,2]
        
        # create dimensions
        dataset.createDimension('latitude',data.shape[0])
        dataset.createDimension('longitude',data.shape[1])
        dataset.createDimension('time',None)
        
        # create some metadata attributes
        dataset.location = location
        dataset.anyattribute = 'Any attribute'
        dataset._FillValue = -9999.
        
        # create variables
        lats = dataset.createVariable('latitude',float,('latitude',), zlib=True,least_significant_digit=5)
        lons = dataset.createVariable('longitude',float,('longitude',), zlib=True,least_significant_digit=5)
        # Assign units attributes to coordinate var data. This attaches a
        # text attribute to each of the coordinate variables, containing the
        # units.
        lats.units = 'degrees_north'
        lons.units = 'degrees_east'
        # write data to coordinate vars.
        lats[:] = lats_out
        lons[:] = lons_out
        # create the pressure and temperature variables 
        change = dataset.createVariable('change',float, ('latitude','longitude'), zlib=True,least_significant_digit=3)
        unc = dataset.createVariable('uncertainty',float,('latitude','longitude'), zlib=True,least_significant_digit=3)
        mask = dataset.createVariable('mask',float,('latitude','longitude'), zlib=True,least_significant_digit=3)    
        # set the units attribute.
        change.units =  'm'
        unc.units = 'm'
        mask.units = ''
        # write data to variables.
        change[:] = change_out
        unc[:] = unc_out
        mask[:] = mask_out
        # close the file
        print(dataset)

# Read the sample data

In [7]:
changes, extent = raster2array("C:\\Users\\jlandman\\Desktop\\glacier_dh_rasterized_fischer et al. 2015\glacier_dh_rasterized_fischer et al. 2015\\dh_0997.asc")
#uncertainty = 
print(extent)

mask, extent = raster2array("C:\\Users\\jlandman\\Desktop\\glacier_dh_rasterized_fischer et al. 2015\glacier_dh_rasterized_fischer et al. 2015\\dh_0997.asc")

print(changes.shape)
dvi=np.dstack((changes,changes, mask))
dvi.shape


[637812.5, 25.0, 157162.5, -25.0]
-3.402823e+38
0.0
(765, 490)


(765, 490, 3)

In [8]:
dvi[...,0][dvi[...,0] != -9999.]

array([ -4.76259995,  -1.26890004, -10.42590046, ...,  -0.70039999,
         3.75399995,   2.53449988], dtype=float32)

In [13]:
make_nc4('C:\\users\\jlandman\\desktop\\test.nc', dvi, extent, location='Switzerland')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    location: Switzerland
    anyattribute: Any attribute
    _FillValue: -9999.0
    dimensions(sizes): latitude(765), longitude(490), time(0)
    variables(dimensions): float64 latitude(latitude), float64 longitude(longitude), float64 change(latitude,longitude), float64 uncertainty(latitude,longitude), float64 mask(latitude,longitude)
    groups: 

